In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import joblib

# Load dataset
file_path = "gym recommendation.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1')

# Drop unnecessary columns
df = df.drop(columns=['ID', 'Fitness Type'])

# Define NLP training data for fitness goals
nlp_data = {
    "Weight Loss": ["lose weight", "fat loss", "burn calories", "get slimmer", "reduce fat"],
    "Weight Gain": ["gain weight", "build muscle", "increase mass", "bulk up", "get stronger"]
}

# Prepare NLP model training data
nlp_texts, nlp_labels = [], []
for label, texts in nlp_data.items():
    nlp_texts.extend(texts)
    nlp_labels.extend([label] * len(texts))

# Train NLP classifier
nlp_model = Pipeline([
    ("vectorizer", TfidfVectorizer()),
    ("classifier", MultinomialNB())
])
nlp_model.fit(nlp_texts, nlp_labels)

# Save NLP model
joblib.dump(nlp_model, "nlp_fitness_goal_model.pkl")

# Function to classify fitness goal using NLP
def predict_fitness_goal(text):
    return nlp_model.predict([text])[0]

# Apply NLP model to classify free-text fitness goals
df['Fitness Goal'] = df['Fitness Goal'].apply(lambda x: predict_fitness_goal(str(x)))

# Encode categorical variables
label_encoders = {}
categorical_columns = ['Sex', 'Hypertension', 'Diabetes', 'Level', 'Fitness Goal', 'Exercises', 'Equipment', 'Diet', 'Recommendation']
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# Splitting data into input (X) and output (y)
X = df[['Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'BMI', 'Level', 'Fitness Goal']]
y = df[['Level', 'Exercises', 'Equipment', 'Diet', 'Recommendation']]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create pipeline for training
pipeline = Pipeline([
    ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42)))
])

# Train model
pipeline.fit(X_train, y_train)

# Save trained model and label encoders
joblib.dump(pipeline, "nlp_fitness_model4.pkl")
joblib.dump(label_encoders, "label_encoders4.pkl")

print("✅ Model updated and saved successfully!")


✅ Model updated and saved successfully!


In [2]:
import requests

url = "http://127.0.0.1:8000/predict"

test_inputs = [
    {
        "Sex": "Male",
        "Age": 25,
        "Height": 175.0,
        "Weight": 70.0,
        "Hypertension": "No",
        "Diabetes": "No",
        "BMI": 22.9,
        "Level": "Beginner",
        "Fitness_Goal": "I want to burn fat"
    },
    {
        "Sex": "Female",
        "Age": 30,
        "Height": 160.0,
        "Weight": 60.0,
        "Hypertension": "No",
        "Diabetes": "Yes",
        "BMI": 23.5,
        "Level": "Intermediate",
        "Fitness_Goal": "I need to gain muscle"
    }
]

for test_input in test_inputs:
    response = requests.post(url, json=test_input)
    print("Input:", test_input)
    print("Response:", response.json())
    print("="*50)


Input: {'Sex': 'Male', 'Age': 25, 'Height': 175.0, 'Weight': 70.0, 'Hypertension': 'No', 'Diabetes': 'No', 'BMI': 22.9, 'Level': 'Beginner', 'Fitness_Goal': 'I want to burn fat'}
Response: {'error': "'Fitness Goal'"}
Input: {'Sex': 'Female', 'Age': 30, 'Height': 160.0, 'Weight': 60.0, 'Hypertension': 'No', 'Diabetes': 'Yes', 'BMI': 23.5, 'Level': 'Intermediate', 'Fitness_Goal': 'I need to gain muscle'}
Response: {'error': "'Fitness Goal'"}
